In [23]:
import pandas as pd
import datetime
import MySQLdb
import os
import warnings
# warnings.filterwarnings("ignore")

In [42]:
db = MySQLdb.connect("localhost","root","123456")
db.set_character_set("gbk")

In [ ]:
db = MySQLdb.connect("47.100.14.137","hyps44413","Nanhua")
db.set_character_set('gbk')

In [44]:
db.close()

In [35]:
dbname = 'sr_oi_option_put'
cursor = db.cursor()

name = '`前海期货`'

sql = "ALTER TABLE " + dbname + ".v_rank ADD COLUMN " + name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".b_rank ADD COLUMN " + name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".s_rank ADD COLUMN " + name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".v_increment ADD COLUMN " + name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".b_increment ADD COLUMN " + name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".s_increment ADD COLUMN " + name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".v_volume ADD COLUMN " + name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".b_oi ADD COLUMN " + name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".s_oi ADD COLUMN " + name + " TEXT"
cursor.execute(sql)

cursor.close()

In [43]:
company_order = pd.read_excel("../期貨公司名稱/鄭商所/sr_company_name.xlsx",header=None)
DATA_DIR = "../各交易所期貨持倉量/鄭商所/更新_期權/"

dbname1 = "sr_oi_option_put"
dbname2 = "sr_oi_option_call"

cursor = db.cursor()
sql = "CREATE DATABASE IF NOT EXISTS " + dbname1 + "_original_data"
cursor.execute(sql)
sql = "CREATE DATABASE IF NOT EXISTS " + dbname2 + "_original_data"
cursor.execute(sql)
sql = "CREATE DATABASE IF NOT EXISTS " + dbname1
cursor.execute(sql)
sql = "CREATE DATABASE IF NOT EXISTS " + dbname2
cursor.execute(sql)

for file in os.listdir(DATA_DIR):
    
    split_name = file.split("_")
    contract_id = split_name[0]
    date = split_name[1].split(".csv")[0]
    
    date = datetime.datetime.strptime(date,'%Y%m%d')   
    
    if (date >= datetime.datetime(2018,1,9)):
        
        df = pd.read_csv(DATA_DIR + file,engine="python",encoding="utf-8-sig")
        date = date.strftime('%Y-%m-%d')
    
        for i in range(len(df)):
            for j in range(10):
                if(str(df.loc[i][j]).strip() == "-"):
                    df.at[i,str(j)] = None  #塞原始資料

                elif((j==2 or j==3 or j==5 or j==6 or j==8 or j==9) and df.loc[i][j] != None):
                    df.at[i,str(j)] = str(df.iloc[i][j]).replace(',','')

        if (contract_id.find("C") != -1):
            databaseName = dbname1
        else:
            databaseName = dbname2


        create_total_volume(databaseName)
        V_info_create(databaseName,company_order)
        B_info_create(databaseName,company_order)
        S_info_create(databaseName,company_order)


        V_info_insert(df, contract_id, date, databaseName)
        B_info_insert(df, contract_id, date, databaseName)
        S_info_insert(df, contract_id, date, databaseName)
        total_volume_insert(df,contract_id,date,databaseName)

    #     total_insert(df,file.split(".csv")[0],databaseName+"_original_data")

C:\Users\Philip\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: Warning: (1007, "Can't create database 'sr_oi_option_put_original_data'; database exists")
  if __name__ == '__main__':
C:\Users\Philip\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: Warning: (1007, "Can't create database 'sr_oi_option_call_original_data'; database exists")
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Philip\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: Warning: (1050, "Table 'total_volume' already exists")
  """
C:\Users\Philip\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: Warning: (1050, "Table 'v_rank' already exists")
  if __name__ == '__main__':
C:\Users\Philip\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: Warning: (1050, "Table 'v_volume' already exists")
  if sys.path[0] == '':
C:\Users\Philip\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: Warning: (1050, "Table 'v_increment' already exists")
  from ipykernel import kernelapp as app
C:\U

In [32]:
b = []
c = []

a = df['1']
a = pd.DataFrame(list(a))

for i in range(len(a) - 1):
    c.append(a.loc[i][0])

for i in range(len(company_order)):
    b.append(company_order.loc[i][0])
    
for x in c:
    if x.strip() not in b:
        print(x)

前海期货        


In [41]:
date,contract_id

('2018-07-06', 'SRC')

In [14]:
def total_insert(data,tbname,databaseName):       
    cursor = db.cursor()
    
    print(tbname)
    
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + "." + tbname + "(\
    Rank INT(5),\
    Member TEXT,\
    Volume FLOAT(5),\
    Increment FLOAT(5),\
    Member1 TEXT,\
    Buy_Volume INT(10),\
    Increment_1 FLOAT(5),\
    Member2 TEXT,\
    Sales_Volume FLOAT(5),\
    Increment_2 FLOAT(5) )"

    cursor.execute(sql)
    value=[]

    for i in range(len(data)-1):      
        value.append((str(data.loc[i][0]).strip(),str(data.loc[i][1]).strip(),str(data.loc[i][2]).strip(),str(data.loc[i][3]).strip(),\
                      str(data.loc[i][4]).strip(),str(data.loc[i][5]).strip(),str(data.loc[i][6]).strip(),str(data.loc[i][7]).strip(),\
                      str(data.loc[i][8]).strip(),str(data.loc[i][9]).strip()))
    
    sql = "INSERT INTO " + databaseName + "." + tbname +" VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"        
    cursor.executemany(sql,value)
    db.commit()
    cursor.close()

In [28]:
def create_total_volume(databaseName):
    cursor = db.cursor()
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".total_volume (Date DATE ,Contract_id TEXT(5), \
    V_Valume FLOAT,V_Increment FLOAT,\
    B_Valume FLOAT,B_Increment FLOAT,\
    S_Valume FLOAT,S_Increment FLOAT)"

    cursor.execute(sql)
    
def total_volume_insert(data, contract_id, table_date, databaseName):
    cursor = db.cursor()
    total_len = len(data)-1
    sql = "INSERT INTO " + databaseName + ".total_volume VALUES(%s,%s,%s,%s,%s,%s,%s,%s)"
    
    if(str(df.loc[total_len][1]) == 'None' or str(df.loc[total_len][1]).strip() == ""):
        cursor.execute(sql,(table_date,contract_id,str(data.loc[total_len][2]).strip(),str(data.loc[total_len][3]).strip(),\
                            str(data.loc[total_len][5]).strip(),str(data.loc[total_len][6]).strip(),\
                            str(data.loc[total_len][8]).strip(),str(data.loc[total_len][9]).strip()))
    else:
        cursor.execute(sql,(table_date,contract_id,str(data.loc[total_len][1]).strip(),str(data.loc[total_len][2]).strip(),\
                            str(data.loc[total_len][4]).strip(),str(data.loc[total_len][5]).strip(),\
                            str(data.loc[total_len][7]).strip(),str(data.loc[total_len][8]).strip()))
        db.commit()
        cursor.close()

In [21]:
def V_info_create(databaseName, company_order):
    cursor = db.cursor()
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' TEXT,'
    name = name + company_order.loc[len(company_order)-1][0] + " TEXT"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".v_rank ( Date DATE, Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".v_volume ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".v_increment ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)
    
    cursor.close()

def V_info_insert(data, contract_name, table_date, databaseName):
    cursor = db.cursor()
    
    empty = ""
    empty = empty + "'" + table_date + "', '" + contract_name + "'"


    sql = "INSERT INTO " + databaseName + ".v_rank (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".v_volume (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".v_increment (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(1,len(data)-1):
        if(data.loc[i][1] == None):
            continue
        
        else:
            sql = "UPDATE " + databaseName + ".v_rank SET " + str(data.loc[i][1]).strip() + "=" + str(data.loc[i][0]).strip() + \
                  " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()    

    for i in range(1,len(data)-1):
        if(data.loc[i][1] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".v_volume SET " + str(data.loc[i][1]).strip() + "=" + str(data.loc[i][2]).strip() + \
                  " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()

    for i in range(1,len(data)-1):
        if(data.loc[i][1] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".v_increment SET " + str(data.loc[i][1]).strip() + "=" + str(data.loc[i][3]).strip() + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()
    
    cursor.close()

## B名次、買單量、增減

def B_info_create(databaseName, company_order):
    cursor = db.cursor()
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' TEXT,'
    name = name + company_order.loc[len(company_order)-1][0] + " TEXT"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_rank ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_oi ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_increment ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)
    cursor.close()

def B_info_insert(data, contract_name, table_date, databaseName):
    cursor = db.cursor()
 
    empty = ""
    empty = empty + "'" + table_date + "', '" + contract_name + "'"
    
    sql = "INSERT INTO " + databaseName + ".b_rank (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".b_oi (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".b_increment (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(1,len(data)-1):
        if(data.loc[i][4] == None):
            continue  
        else:
            sql = "UPDATE " + databaseName + ".b_rank SET " + str(data.loc[i][4]).strip() + "=" + str(data.loc[i][0]).strip() + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()    

    for i in range(1,len(data)-1):
        if(data.loc[i][4] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".b_oi SET " + str(data.loc[i][4]).strip() + "=" + str(data.loc[i][5]).strip() + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()

    for i in range(1,len(data)-1):
        if(data.loc[i][4] == None):
            continue
        
        else:
            sql = "UPDATE " + databaseName + ".b_increment SET " + str(data.loc[i][4]).strip() + "=" + str(data.loc[i][6]).strip() + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()

    cursor.close()

## S名次、賣單量、增減

def S_info_create(databaseName, company_order):
    cursor = db.cursor()
    name=""
    
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' TEXT,'
    name = name + company_order.loc[len(company_order)-1][0] + " TEXT"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".s_rank ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".s_oi ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".s_increment ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)
    cursor.close()

def S_info_insert(data, contract_name, table_date, databaseName):
    cursor = db.cursor()

    empty = ""
    empty = empty + "'" + table_date + "', '" + contract_name + "'"
    
    sql = "INSERT INTO " + databaseName + ".s_rank (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".s_oi (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".s_increment (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(1,len(data)-1):
        if(data.loc[i][7] == None):
            continue  
        
        else:
            sql = "UPDATE " + databaseName + ".s_rank SET " + str(data.loc[i][7]).strip() + "=" + str(data.loc[i][0]).strip() + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()    

    for i in range(1,len(data)-1):
        if(data.loc[i][7] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".s_oi SET " + str(data.loc[i][7]).strip() + "=" + str(data.loc[i][8]).strip() + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()

    for i in range(1,len(data)-1):
        if(data.loc[i][7] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName +".s_increment SET " + str(data.loc[i][7]).strip() + "=" + str(data.loc[i][9]).strip() + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()

    cursor.close()